In [1]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import logging
import pickle
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial
import gc
from importlib import reload
import pyreadstat
import disease_network_funs as fun_py
import re

In [20]:
reload(fun_py)

<module 'disease_network_funs' from '/home/hashjamm/codes/disease_network/disease_network_funs.py'>

In [3]:
sas_path = "/home/hashjamm/project_data/disease_network/sas_files/"
edge_pids_path = "/home/hashjamm/results/disease_network/edge_pids/"
matched_path = "/home/hashjamm/project_data/disease_network/sas_files/matched/"
ctable_path = "/home/hashjamm/results/disease_network/ctables/"
pids_info_path = "/home/hashjamm/results/disease_network/"
final_results_path = "/home/hashjamm/results/disease_network/final_results/"

In [8]:
node_pids_info = pd.read_csv(f'{pids_info_path}node_pids_info.csv')

In [18]:
node_pids_info.drop(columns=['width', 'height']).head()

,node_code,sex_1_counts,sex_2_counts,age_0_counts,age_1_counts,age_2_counts,age_3_counts,age_4_counts,age_5_counts,age_6_counts,...,sex_2_ctrb_3_counts,sex_2_ctrb_4_counts,sex_2_ctrb_5_counts,sex_2_ctrb_6_counts,sex_2_ctrb_7_counts,sex_2_ctrb_8_counts,sex_2_ctrb_9_counts,sex_2_ctrb_10_counts,Korean,English
0,J03,41118,51040,1227,7231,11844,7787,5556,6026,7075,...,3706,4041,4719,5337,6200,6945,6932,6680,급성 편도염,Acute tonsillitis
1,J20,33557,44137,1392,7547,8334,5408,3702,3911,5102,...,3076,3551,4110,4759,5237,6008,5889,5823,급성 기관지염,Acute bronchitis
2,J06,30228,39585,1318,7861,7637,4603,3452,3802,4751,...,2858,3271,3662,4201,4824,5166,5258,5252,다발성 및 상세불명 부위의 급성 상기도 감염,Acute upper respiratory infections of muitiple...
3,J02,27607,36329,1561,7785,7644,4575,3204,3611,4415,...,2446,2923,3262,3855,4456,4892,5164,4934,급성 인두염,Acute pharyngitis
4,J00,27512,35232,1308,7750,6915,4203,3031,3233,3786,...,2525,2972,3344,3615,4196,4626,4698,4474,급성 비인두염[감기],Acute nasopharyngitis[common cold]


In [25]:
# cause_pids_info = fun_py.colname_type_setting(node_pids_info, 1, 'cause')
# 기존에는 node_pids_info 에서 가로와 세로 길이, 그리고 한국어명 영어명명에 대한 정보는 없었던 것으로 보임. 
# 이 시점 뒤에 어느 시점에서 붙은 것 같은데, 정확하게 어디에서 붙은지 알 수 없어서 그걸 때어내는 코드를 일부러 적어서 진행
cause_pids_info = fun_py.colname_type_setting(node_pids_info.drop(columns=['width', 'height', 'Korean', 'English']), 1, 'cause')
cause_pids_info = cause_pids_info.rename(columns={'node_code':'cause_abb'})

In [26]:
outcome_pids_info = fun_py.colname_type_setting(node_pids_info.drop(columns=['width', 'height', 'Korean', 'English']), 1, 'outcome')
outcome_pids_info = outcome_pids_info.rename(columns={'node_code':'outcome_abb'})

In [27]:
for i in range(1, 11):
    edge_pids_info = pd.read_csv(f'{pids_info_path}edge_pids_info_{i}.csv')
    stat_cut_result = pd.read_csv(f'{ctable_path}stat_cut_result_{i}.csv')
    
    edge_pids_info = fun_py.colname_type_setting(edge_pids_info, 2, 'edge')
    
    final_result =\
    pd.merge(stat_cut_result, edge_pids_info, on=['cause_abb', 'outcome_abb'], how='inner').reset_index(drop=True)
    
    final_result =\
    pd.merge(final_result, cause_pids_info, on=['cause_abb'], how='inner').reset_index(drop=True)
    
    final_result =\
    pd.merge(final_result, outcome_pids_info, on=['outcome_abb'], how='inner').reset_index(drop=True)
    
    cis_cut_final_result = final_result[final_result['rr_lower_cis'] > 1]
    
    final_result.to_csv(f"{final_results_path}final_result_{i}.csv", index=False)
    cis_cut_final_result.to_csv(f"{final_results_path}cis_cut_final_result_{i}.csv", index=False)